### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []
for page_num in range(1, 350):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    
    page_url = f"https://chinhsachonline.chinhphu.vn/chinh-sach-voi-nguoi-co-cong/t1/2/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_chinhsach = pd.DataFrame(articles_data)
df_chinhsach.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Văn Dũng,08:10 08/08/2024,Vợ liệt sĩ tái giá có được chế độ mai táng phí?,"Tôi xin hỏi, vợ liệt sĩ tái giá, hưởng trợ cấp...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
1,Ngọc Huyền,15:32 02/07/2024,Người làm nghĩa vụ quốc tế có được giảm tiền s...,"Bố tôi là cựu chiến binh, thực hiện nghĩa vụ q...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
2,Trần Anh Túc,16:53 28/06/2024,Căn cứ xác định người có công giúp đỡ cách mạng,Theo quy định tại Điều 38 Pháp lệnh Ưu đãi ngư...,Bộ Lao động – Thương binh và Xã hội trả lời vấ...
3,Nguyễn Đức Lộc,07:05 09/06/2024,Trường hợp nào được giảm 80% tiền sử dụng đất?,Tôi là người hoạt động kháng chiến bị nhiễm ch...,Bộ Lao động – Thương binh và Xã hội trả lời vấ...
4,Nguyễn Thị Tân,09:55 07/06/2024,Điều kiện thân nhân thương binh được trợ cấp t...,"Chồng tôi tham gia cách mạng, là thương binh l...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../data/data_tinhhuong/chinhsach_nguoicocong_data.csv'
df_chinhsach.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/data_tinhhuong/chinhsach_nguoicocong.csv


In [6]:
df_chinhsach=pd.read_csv("../data/data_tinhhuong/chinhsach_nguoicocong_data.csv",encoding='utf-8')

In [7]:
df_chinhsach.head(5)

,Index,Name,Time,Question,Situation,Answer
0,0,Nguyễn Văn Dũng,08:10 08/08/2024,Vợ liệt sĩ tái giá có được chế độ mai táng phí?,"Tôi xin hỏi, vợ liệt sĩ tái giá, hưởng trợ cấp...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
1,1,Ngọc Huyền,15:32 02/07/2024,Người làm nghĩa vụ quốc tế có được giảm tiền s...,"Bố tôi là cựu chiến binh, thực hiện nghĩa vụ q...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
2,2,Trần Anh Túc,16:53 28/06/2024,Căn cứ xác định người có công giúp đỡ cách mạng,Theo quy định tại Điều 38 Pháp lệnh Ưu đãi ngư...,Bộ Lao động – Thương binh và Xã hội trả lời vấ...
3,3,Nguyễn Đức Lộc,07:05 09/06/2024,Trường hợp nào được giảm 80% tiền sử dụng đất?,Tôi là người hoạt động kháng chiến bị nhiễm ch...,Bộ Lao động – Thương binh và Xã hội trả lời vấ...
4,4,Nguyễn Thị Tân,09:55 07/06/2024,Điều kiện thân nhân thương binh được trợ cấp t...,"Chồng tôi tham gia cách mạng, là thương binh l...",Bộ Lao động – Thương binh và Xã hội trả lời vấ...
